# QCTO - Workplace Module

### Investigating and Predicting Landslide Risk in Iran
#### Done By: Mieke Spaans

© ExploreAI 2024

---
 <a id="cont"></a>
## Table of Contents

<a href=#BC> Background Context</a>

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Data Collection and Description</a>

<a href=#three>3. Loading Data </a>

<a href=#four>4. Data Cleaning and Filtering</a>

<a href=#five>5. Exploratory Data Analysis (EDA)</a>

<a href=#six>6. Modeling </a>

<a href=#seven>7. Evaluation and Validation</a>

<a href=#eight>8. Final Model</a>

<a href=#nine>9. Conclusion and Future Work</a>

<a href=#ten>10. References</a>

---
 <a id="BC"></a>
## **Background Context**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Introduce the project, outline its goals, and explain its significance.
* **Details:** Include information about the problem domain, the specific questions or challenges the project aims to address, and any relevant background information that sets the stage for the work.
---

Add introduction here

---
<a id="one"></a>
## **Importing Packages**
<a href=#cont>Back to Table of Contents</a>

---


**Purpose**: Set up the Python environment with necessary libraries and tools.

Listed below is all of the Python packages that will be used throughout the project, such as Pandas for data manipulation, Matplotlib/Seaborn for visualization, scikit-learn for modeling, etc. I have seperated the code blocks to libraries used for specific parts of the project for ease of reading and debugging.

In [13]:
# MLFlow is used to perform model hyperparameter tuning
#!pip install mlflow

In [14]:
# loading .csv files via google drive:
#from google.colab import drive

In [43]:
# Libraries used for data loading and pre-processing:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [16]:
# Libraries used for data visualisation during exploratory data analysis:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from scipy import stats
import geopandas as gpd
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.image as mpimg
import contextily as ctx

In [17]:
# Model training libraries:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
# Model evaluation libraries:
from sklearn.metrics import mean_squared_error

---
<a id="two"></a>
## **Data Collection and Description**
<a href=#cont>Back to Table of Contents</a>

---

### Data Collection:
The dataset I am using for this project is publicly available on Kaggle.

Link to the Kaggle page that contains the .csv file: https://www.kaggle.com/datasets/mohammadrahdanmofrad/landslide-risk-assessment-factors

The Kaggle page mentions that it was sourced from 'Topography Maps and Google Erath'[sic]. Nothing further is mentioned regarding sourcing data. The dataset author, Mohamad Rahdan, is a Geospatial Data Scientist.

The data is licensed to the CC BY-SA 4.0, or the Creative Commons Attribution-ShareAlike 4.0 International. This means I may:
- Share — copy and redistribute the material in any medium or format for any purpose, even commercially.
- Adapt — remix, transform, and build upon the material for any purpose, even commercially.

Please note, however, according to this license I must comply to the following rules:
- Attribution — You must give appropriate credit , provide a link to the license, and indicate if changes were made . You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
- ShareAlike — If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.
- No additional restrictions — You may not apply legal terms or technological measures that legally restrict others from doing anything the license permits.

The above two paragraphs were copied directly off the Creative Commons website, linked below.
https://creativecommons.org/licenses/by-sa/4.0/

### Data Description:
The dataset contains over 4000 landslide hazard points in Iran. Each point includes longitude and latitude information to help locate the point on a map.

##### Definition: "Landslide point":

A "Landslide Point" refers to a specific geographical location that has been identified as having a heightened risk of experiencing landslides. This designation is typically based on various factors such as the area's slope, soil composition, rainfall patterns, and other environmental conditions that contribute to instability and susceptibility to landslides. Landslide points are often determined through analysis of historical landslide data, geological surveys, and climate assessments, and are used to assess and mitigate risks in vulnerable regions.

##### The dataset contains 15 columns: [descriptions taken directly off the Kaggle page]
- `ID`
- `LONG`: The longitude of the landslide point
- `LAT`: The latitude of the landslide point
- `SUB_Basin`: The name of the watershed of the landslide point
- `Elevation`: The elevation of the landslide point from sea level (m)
- `AAP(mm)`: The average annual precipitation at the landslide point
- `RiverDIST(m)`: The distance of the landslide point from the river
- `FaultDIST(m)`: The distance of the landslide point from the fault
- `Landuse_Type`: The landuse type at the landslide point
- `Slope(Percent)`: The slope at the landslide point (Values range from 0 to 100%)
- `Slope(Degrees)`: The slope at the landslide point (Values range from 0 to 90)
- `GEO_UNIT`: The geology unit of the landslide point
- `DES_GEOUNI`: The description of the geology unit of the landslide point
- `Climate_Type`: The climate type of the landslide point
- `DES_ClimateType`: The description of the climate type of the landslide point

All but six columns are numerical. The exceptions are:
- `SUB_Basin`
- `Landuse_Type`
- `GEO_UNIT`
- `DES_GEOUNI`
- `Climate_Type`
- `DES_ClimateType`

All are instances of text data.

##### First impressions:
This dataset is very thorough, as each landslide point contains a large variety of different factors, like distance to river and fault, slope, geology unit, etc.

One major issue is the fact that this dataset focuses on landslide points in Iran. For this project to provide real-world benefit, the dataset would be better to contain a variety of countries' landslide risk points. Despite this setback, a workaround would be to use the information gathered here and apply it to countries and regions of similar climate and topographical setting.

With regards to model building:
- We can look into classification, where we would input a new location and determine if it is a possible landslide point. This would be valuable to predict possible landslide points.
- We could also look into unsupervised learning, by applying KNN clustering (or something similar) to the landslide points. It would enable us to understand different categories of landslides. This could help researchers.

---
<a id="three"></a>
## **Loading Data**
<a href=#cont>Back to Table of Contents</a>
---

In [19]:
# Loading via pandas when using Jupyter Notebooks
landslide = pd.read_csv('Landslide_Factors_IRAN.csv')

In [20]:
# Check that the data was imported correctly
landslide.head(3)

,ID,LONG,LAT,SUB_Basin,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Landuse_Type,Slop(Percent),Slop(Degrees),GEO_UNIT,DES_GEOUNI,Climate_Type,DES_ClimateType
0,1,52.326,27.763,Mehran,617.0,137,1448.705292,40639.57890,poorrange,42.240669,22.899523,EOas-ja,"Undivided Asmari and Jahrum Formation , regard...",A-M-VW,"Warm and humid, with a humid period longer tha..."
1,2,52.333,27.772,Mehran,944.0,137,344.299484,40135.02913,mix(woodland_x),68.219116,34.301464,KEpd-gu,Keewatin Epedotic quartz diorite,A-M-VW,"Warm and humid, with a humid period longer tha..."
2,3,52.326,27.763,Mehran,617.0,137,1448.705292,40639.57890,poorrange,42.240669,22.899523,EOas-ja,"Undivided Asmari and Jahrum Formation , regard...",A-M-VW,"Warm and humid, with a humid period longer tha..."


From the above code, we see that the dataframe imports correctly.

Some issues noted:
- `GEO_UNIT` is in code form and difficult to understand.
- `Climate_Type` is also in code form and difficult to understand.
- `DES_ClimateType` repeats. Consider removing it and only relying on `Climate_Type`.
- `ID` was loaded as a column, but functions as index. Need to either remove or replace existing index.

In [21]:
# Check size of dataframe
landslide.shape

(4295, 15)

The dataframe contains 4295 entries, and has 15 columns.

In [22]:
# Get each column's data type, and count non-null values
landslide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4295 entries, 0 to 4294
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               4295 non-null   int64  
 1   LONG             4295 non-null   float64
 2   LAT              4295 non-null   float64
 3   SUB_Basin        4295 non-null   object 
 4   Elevation        4295 non-null   float64
 5   AAP(mm)          4295 non-null   int64  
 6   RiverDIST(m)     4295 non-null   float64
 7   FaultDIST(m)     4295 non-null   float64
 8   Landuse_Type     4295 non-null   object 
 9   Slop(Percent)    4295 non-null   float64
 10  Slop(Degrees)    4295 non-null   float64
 11  GEO_UNIT         4295 non-null   object 
 12  DES_GEOUNI       4295 non-null   object 
 13  Climate_Type     4295 non-null   object 
 14  DES_ClimateType  4295 non-null   object 
dtypes: float64(7), int64(2), object(6)
memory usage: 503.4+ KB


From this we find out what each column's data type is, as well as the total number of non-null values in each.

Analysis:
- The three data types present are:
    - Integer (2 columns)
    - Float (7 columns)
    - Object (typically string values) (6 columns)
- No column contains null values, which makes our job a little easier in the next phase.

---
<a id="four"></a>
## **Data Cleaning and Filtering**
<a href=#cont>Back to Table of Contents</a>

---

**Purpose:** Prepare the data for analysis by cleaning and filtering.

**Details:** Steps detailed for handling missing values and removing outliers.

### Checking for null values

In [23]:
# Checking for null values:
landslide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4295 entries, 0 to 4294
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               4295 non-null   int64  
 1   LONG             4295 non-null   float64
 2   LAT              4295 non-null   float64
 3   SUB_Basin        4295 non-null   object 
 4   Elevation        4295 non-null   float64
 5   AAP(mm)          4295 non-null   int64  
 6   RiverDIST(m)     4295 non-null   float64
 7   FaultDIST(m)     4295 non-null   float64
 8   Landuse_Type     4295 non-null   object 
 9   Slop(Percent)    4295 non-null   float64
 10  Slop(Degrees)    4295 non-null   float64
 11  GEO_UNIT         4295 non-null   object 
 12  DES_GEOUNI       4295 non-null   object 
 13  Climate_Type     4295 non-null   object 
 14  DES_ClimateType  4295 non-null   object 
dtypes: float64(7), int64(2), object(6)
memory usage: 503.4+ KB


As discussed in the previous part (Loading Data) we see that there are no null values present.

### Removing the column `ID`, as it's values coincide with the index

In [24]:
# Removing the column ID, as it just repeats the index
landslide.drop('ID',axis= 1 ,inplace= True)
landslide.head()

,LONG,LAT,SUB_Basin,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Landuse_Type,Slop(Percent),Slop(Degrees),GEO_UNIT,DES_GEOUNI,Climate_Type,DES_ClimateType
0,52.326,27.763,Mehran,617.0,137,1448.705292,40639.57890,poorrange,42.240669,22.899523,EOas-ja,"Undivided Asmari and Jahrum Formation , regard...",A-M-VW,"Warm and humid, with a humid period longer tha..."
1,52.333,27.772,Mehran,944.0,137,344.299484,40135.02913,mix(woodland_x),68.219116,34.301464,KEpd-gu,Keewatin Epedotic quartz diorite,A-M-VW,"Warm and humid, with a humid period longer tha..."
2,52.326,27.763,Mehran,617.0,137,1448.705292,40639.57890,poorrange,42.240669,22.899523,EOas-ja,"Undivided Asmari and Jahrum Formation , regard...",A-M-VW,"Warm and humid, with a humid period longer tha..."
3,52.333,27.694,Mehran,55.0,137,1889.828623,42189.54442,rock,12.141766,6.922833,Mlmmi,Low weathering grey marls alternating with ba...,A-M-VW,"Warm and humid, with a humid period longer tha..."
4,52.324,27.682,Mehran,20.0,137,874.201691,43010.08400,poorrange,2.216230,1.269598,MuPlaj,"Brown to grey , calcareous , feature - formin...",A-M-VW,"Warm and humid, with a humid period longer tha..."


### Checking the text columns:


The text data we are working with is related to locations `SUB_Basin`, types of land use `Landuse_Type` and the geology and climate of the area. This text data is short, and often just one word. With this in mind, I first check `SUB_Basin`'s unique values, to make sure they are consistent.

#### Checking SUB_Basin:

In [25]:
landslide['SUB_Basin'].unique()

array(['Mehran', 'Mand', 'Shoor', 'Rudan', 'Iran Southern Coast',
       'Jaghin', 'Halil Rud', 'Rakshan', 'Dalaki', 'Qarabaq', 'Zohreh',
       'Neyriz Lake', 'Namakzar-e Shahdad', 'Shur Ab', 'Bampur', 'Karun',
       'Zal', 'Marun', 'Kor', 'Zayandeh Rud', 'Shadkam', 'Gamasb',
       'Nahr at Tib', 'Karkheh', 'Dez', 'Dasht-e Kavir', 'Upper Hari Rod',
       'Nahrawan Canal', 'Diyala', 'Masileh', 'Kavir-o Namak',
       'Simineh Rud', 'Lesser Zab', 'Safid Rud', 'Zarineh Rud', 'Shur',
       'Qezel Owzan', 'Se Hezar', 'Chalus', 'Talar', 'Babol', 'Jajrood',
       'Haraz', 'Hableh Rud', 'Karaj', 'Qarasu', 'Neka', 'Tajan',
       'Haj Ali Gholi Salt Lake', 'Tedzhen', 'Nakhchivan Reservoir',
       'Lake Orumieh', 'Talkheh Rud', 'Karasu', 'Safid Rud Delta',
       'Vilascay / Lankaran / Tangarud', ' ', 'Gorgan', 'Rud-e Atrak',
       'Atrak Delta', 'Kara-Kum Desert', 'Sumbar', 'Ararat', 'Aras 2',
       'Lenkeran'], dtype=object)

From this we gather:
- The unique terms follow the same style, they are capitalised and terms are seperated by a space.
- One of the values is ' ', which would indicate the landslide point is not associated with a river basin. We investigate this further in the code block below.
- One of the values is 'Aras 2'. A quick Google search reveals that the Aras-2 is a military vehicle. Aras is also a river in the Caucasus. There is no mention of a 'Aras 1', will replace with 'Aras'.

In [26]:
# Checking where SUB_Basin is empty
landslide[landslide['SUB_Basin'] == ' ']

,LONG,LAT,SUB_Basin,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Landuse_Type,Slop(Percent),Slop(Degrees),GEO_UNIT,DES_GEOUNI,Climate_Type,DES_ClimateType
3551,50.458,37.264,,0.0,825,17401.6644,29964.55476,,0.0,0.0,,,,


The above row is the only row whose text columns are empty. We remove this row.

In [27]:
# Remove row mentioned above
landslide.drop(index= 3551, inplace= True)

In [28]:
# Replacing Aras 2 with Aras
landslide.replace(to_replace= 'Aras 2', value= 'Aras', inplace= True)

In [29]:
# Check that two changes worked
landslide['SUB_Basin'].unique()

array(['Mehran', 'Mand', 'Shoor', 'Rudan', 'Iran Southern Coast',
       'Jaghin', 'Halil Rud', 'Rakshan', 'Dalaki', 'Qarabaq', 'Zohreh',
       'Neyriz Lake', 'Namakzar-e Shahdad', 'Shur Ab', 'Bampur', 'Karun',
       'Zal', 'Marun', 'Kor', 'Zayandeh Rud', 'Shadkam', 'Gamasb',
       'Nahr at Tib', 'Karkheh', 'Dez', 'Dasht-e Kavir', 'Upper Hari Rod',
       'Nahrawan Canal', 'Diyala', 'Masileh', 'Kavir-o Namak',
       'Simineh Rud', 'Lesser Zab', 'Safid Rud', 'Zarineh Rud', 'Shur',
       'Qezel Owzan', 'Se Hezar', 'Chalus', 'Talar', 'Babol', 'Jajrood',
       'Haraz', 'Hableh Rud', 'Karaj', 'Qarasu', 'Neka', 'Tajan',
       'Haj Ali Gholi Salt Lake', 'Tedzhen', 'Nakhchivan Reservoir',
       'Lake Orumieh', 'Talkheh Rud', 'Karasu', 'Safid Rud Delta',
       'Vilascay / Lankaran / Tangarud', 'Gorgan', 'Rud-e Atrak',
       'Atrak Delta', 'Kara-Kum Desert', 'Sumbar', 'Ararat', 'Aras',
       'Lenkeran'], dtype=object)

#### Check Landuse_Type:

In [30]:
# Check other text data:
landslide['Landuse_Type'].unique()

array(['poorrange', 'mix(woodland_x)', 'rock', 'mix(agri_follow)',
       'mix(bareland_x)', 'verylowforest', 'lowforest', 'woodland1',
       'bareland', 'modrange', 'mix(poorrange_x)', 'mix(lowforest_x)',
       'agri', 'masil', 'modforest', 'mix(modforest_x)',
       'mix(dryfarming_x)', 'mix(modrange_x)', 'mix(agri_X)',
       'bagh/grazing area', 'goodrange', 'woodland2', 'urban',
       'mix(bagh_X)', 'mix(agri_dryfarming)', 'denseforest',
       'mix(agri_bagh)', 'dryfarming', 'follow', 'mix(goodrang_x)',
       'mix(follow_x)', 'saltland', 'mix(verylowforest_x)', 'wetland1',
       'water', ' ', 'mix(denseforest_x)'], dtype=object)

Looking at unique values for the `Landuse_Type` we gather:
- There are also empty values present, as with the `SUB_Basin` column
- The formatting is all lowercase, with no spaces present between words.
- The term mix(woodland_x) is assumed to indicated a mixed use area. This is difficult to interpret. Will remove the '_x', as the 'mix()' already indicates mix use. I assume the 'x' refers to the secondary land use that is not included in the dataset.
- mix(bagh_X) and mix(agri_X) contains capital X, does not follow formatting of other values. Will remove _X
- 'bagh/grazing area' does not follow formatting convention as well, will replace with 'mix(bagh_grazing)'

In [31]:
# Check empty values
landslide[landslide['Landuse_Type'] == ' ']

,LONG,LAT,SUB_Basin,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Landuse_Type,Slop(Percent),Slop(Degrees),GEO_UNIT,DES_GEOUNI,Climate_Type,DES_ClimateType
3486,48.660,38.392,Vilascay / Lankaran / Tangarud,290.0,779,14.452563,3698.394077,,5.070903,2.902927,PlQc,Fluvial conglomerate,PH-C-W,"Warm and dry, with a dry period longer than th..."
3913,55.143,37.951,Atrak Delta,62.0,134,45.078001,10598.602870,,4.674369,2.676268,,,A-C-W,"Hot and dry, with a dry period longer than the..."
3975,54.700,37.641,Atrak Delta,4.0,133,7297.298469,25151.376910,,0.380080,0.217769,,,,


Something to consider for future projects:

Find closest longitude and latitude points, and replace empty values according to the closest landslide point.

For the purpose of this project, we just remove these values.

In [32]:
# Removerow mentioned above
landslide.drop(index= [3913, 3975, 3486], inplace= True)

In [33]:
# Update columns so that woodland1 and woodland2 are now woodland:
landslide['Landuse_Type'].replace(to_replace= 'woodland1', value= 'woodland', inplace= True)
landslide['Landuse_Type'].replace(to_replace= 'woodland2', value= 'woodland', inplace= True)
# Update columns so that wetland1 is now wetland:
landslide['Landuse_Type'].replace(to_replace= 'wetland1', value= 'wetland', inplace= True)

Some of the values are too specific, and doesn't add any new information to the data. For the sake of meaningful and clear categories, we create a new column, `Landuse_Category`, with the following values changed:
- Any values that is a mix, but just the first land use is mentioned.
    - examples: mix(woodland_x), mix(bareland_x)
    - change to: woodland, bareland
- verylowforest, lowforest, modforest, denseforest to just "forest"
- modrange, poorrange, goodrange to variousrange
- dryfarming, mix(agri_bagh), etc. to just "agri"

In [34]:
landslide['Landuse_Category'] = landslide['Landuse_Type']
landslide.head(2)

,LONG,LAT,SUB_Basin,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Landuse_Type,Slop(Percent),Slop(Degrees),GEO_UNIT,DES_GEOUNI,Climate_Type,DES_ClimateType,Landuse_Category
0,52.326,27.763,Mehran,617.0,137,1448.705292,40639.57890,poorrange,42.240669,22.899523,EOas-ja,"Undivided Asmari and Jahrum Formation , regard...",A-M-VW,"Warm and humid, with a humid period longer tha...",poorrange
1,52.333,27.772,Mehran,944.0,137,344.299484,40135.02913,mix(woodland_x),68.219116,34.301464,KEpd-gu,Keewatin Epedotic quartz diorite,A-M-VW,"Warm and humid, with a humid period longer tha...",mix(woodland_x)


In [35]:
# Replace bagh/grazing area and dryfarming with "agri"
landslide['Landuse_Category'].replace(to_replace= 'bagh/grazing area', value= 'agri', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'dryfarming', value= 'agri', inplace= True)

# Replace _forest with "forest"
landslide['Landuse_Category'].replace(to_replace= 'verylowforest', value= 'forest', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'lowforest', value= 'forest', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'modforest', value= 'forest', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'denseforest', value= 'forest', inplace= True)

# Replace _range with "variousrange"
landslide['Landuse_Category'].replace(to_replace= 'goodrange', value= 'variousrange', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'modrange', value= 'variousrange', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'poorrange', value= 'variousrange', inplace= True)

# Replace values with mix(_x)
landslide['Landuse_Category'].replace(to_replace= 'mix(agri_follow)', value= 'agri', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(denseforest_x)', value= 'forest', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(verylowforest_x)', value= 'forest', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(follow_x)', value= 'follow', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(goodrang_x)', value= 'variousrange', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(agri_bagh)', value= 'agri', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(agri_dryfarming)', value= 'agri', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(bagh_X)', value= 'bagh', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(agri_X)', value= 'agri', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(modrange_x)', value= 'variousrange', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(dryfarming_x)', value= 'agri', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(modforest_x)', value= 'forest', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(lowforest_x)', value= 'forest', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(poorrange_x)', value= 'variousrange', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(woodland_x)', value= 'woodland', inplace= True)
landslide['Landuse_Category'].replace(to_replace= 'mix(bareland_x)', value= 'bareland', inplace= True)

#### Check GEO_UNIT

In [36]:
landslide['GEO_UNIT'].unique()

array(['EOas-ja', 'KEpd-gu', 'Mlmmi', 'MuPlaj', 'OMr', 'Plbk', 'Qft2',
       'Mlgs', 'Ek', 'tm', 'Qft1', 'Pzkb', 'Eja', 'OMf', 'ElPE-Eh',
       'PlQc', 'TRKurl', 'Ktb', 'OMgr', 'Mur', 'Eabvt', 'E2m', 'pgr',
       'Edv', 'E2s', 'Edavt', 'Edsv', 'Eav', 'Murc', 'K1c', 'Plav', 'sr',
       'PLgr', 'Qabvs', 'Kbgp', 'PeEpd', 'OMas', 'Kgu', 'JKkgp', 'Klsol',
       'KPeam', 'Pec', 'Qtr', 'Jugn', 'OMrb', 'Ktl', 'Pj', 'K1l',
       'EOa-bv', 'Eavt', 'Jh', 'TRsh', 'TRJs', 'Qsf', 'Czl', 'EMas-sb',
       'Ekn', 'MPlfgp', 'KEpd-g', 'E', 'Cm', 'TRkk-nz', 'JKbl', 'Egr',
       'pCk', 'TRn', 'pC-Cs', 'Edt', 'Jel', 'pCrr', 'Dp', 'Cz', 'pCr',
       'Qcf', 'pCmt2', 'PeEf', 'Pld.asv', 'Kus', 'sp', 'pd', 'OMa.bv',
       'OMql', 'Kussh', 'Ebv', 'Ogb', 'KPef', 'KPegr-di', 'TRJvm', 'h',
       'Jph', 'K1bl', 'Pda', 'K1a.bv', 'OMq', 'Ea.bvt', 'MPa.bvt',
       'Odi-gb', 'PLd.av', 'OMml', 'E2f', 'Kgr', 'Olm,s,c', 'COm', 'Judi',
       'Murm', 'K2m,1', 'OMat', 'OMd.av', 'Ktzl', 'TRjs', 'Qm', 'Pd',
       '

This column presents the geology features present in a landslide point. Attempting to decode this without the accompaning `DES_GEOUNI` is futile. With this in mind, I decided not to remove either of the two columns. `DES_GEOUNI` helps us to understand the geology features, and `GEO_UNIT` is easier to perform calculations on.

Features of interest:
- The formatting is not immediately apparent, as there is a combination of capital and lowercase letters. Numbers are also present.
- Due to my limited knowledge in geology, I have elected to leave these codes as is
- I will still update the empty fields, either by removing the rows or imputing values.

In [37]:
landslide[(landslide['GEO_UNIT'] == ' ')]

,LONG,LAT,SUB_Basin,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Landuse_Type,Slop(Percent),Slop(Degrees),GEO_UNIT,DES_GEOUNI,Climate_Type,DES_ClimateType,Landuse_Category
3414,48.595,38.431,Vilascay / Lankaran / Tangarud,1160.0,779,264.940436,3855.000680,denseforest,52.345280,27.629934,,,SH-K-W,"Hot and dry, with a dry period longer than the...",forest
3415,48.595,38.426,Vilascay / Lankaran / Tangarud,1126.0,779,409.951214,3302.061843,mix(goodrang_x),38.172058,20.892885,,,SH-K-W,"Hot and dry, with a dry period longer than the...",variousrange
3417,48.595,38.426,Vilascay / Lankaran / Tangarud,1126.0,779,409.951214,3302.061843,mix(goodrang_x),38.172058,20.892885,,,SH-K-W,"Hot and dry, with a dry period longer than the...",variousrange
3418,48.583,38.433,Vilascay / Lankaran / Tangarud,1426.0,779,1289.269294,4166.959218,mix(goodrang_x),29.620060,16.499321,,,SH-K-M,"Hot and dry, with a dry period longer than the...",variousrange
3485,48.586,38.426,Vilascay / Lankaran / Tangarud,1269.0,779,1133.395937,3370.142858,mix(goodrang_x),49.192505,26.193729,,,SH-K-M,"Hot and dry, with a dry period longer than the...",variousrange
4261,47.020,39.163,Aras,241.0,471,512.715681,9585.855194,mix(agri_bagh),5.912662,3.383766,,,SA-C-W,"Hot and dry, with a dry period longer than the...",agri
4287,48.594,38.442,Vilascay / Lankaran / Tangarud,1519.0,779,128.052600,5079.026924,denseforest,65.444481,33.202534,,,,,forest


We notice that a majority of the empty fields are related to landslide points near 'Vilascay / Lankaran / Tangarud'. I will check this basin, and see if there is an overarching geology type in the area.

In [38]:
landslide['GEO_UNIT'][(landslide['SUB_Basin'] == 'Vilascay / Lankaran / Tangarud')]

3405    KPAavs
3406    KPAavs
3414          
3415          
3417          
3418          
3471      PlQc
3472      PlQc
3482       Qtr
3483       E1f
3484       Eav
3485          
3487      PlQc
3497       Qtr
3498    KPAavs
3511       Eav
3517       K2c
3518    KPAavs
3520    KPAavs
3527    KPAavs
3528        Ku
3529    KPAavs
3533    KPAavs
4287          
4291       Qtr
4294       Qtr
Name: GEO_UNIT, dtype: object

The type 'KPavs' is the most prevalent, but we cannot replace the empty values with 'KPavs'. There are too many different types of `GEO_UNIT` to be certain. In stead, we remove these values.

In [39]:
# remove all empty rows of GEO_UNIT
landslide.drop(index= [3414, 3415, 3417, 3418, 3485, 4261, 4287], inplace= True)

#### Check DES_GEOUNI
This column contains longer strings of text. For this column, I will only look into replacing empty values.

In [40]:
landslide[landslide['DES_GEOUNI'] == ' ']

,LONG,LAT,SUB_Basin,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Landuse_Type,Slop(Percent),Slop(Degrees),GEO_UNIT,DES_GEOUNI,Climate_Type,DES_ClimateType,Landuse_Category


This column is clear.

### Checking numerical columns

In [41]:
landslide.describe()

,LONG,LAT,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Slop(Percent),Slop(Degrees)
count,4284.000000,4284.000000,4284.000000,4284.000000,4284.000000,4284.000000,4284.000000,4284.000000
mean,50.994732,34.926284,1392.764173,241.150093,804.577993,5454.380436,23.075443,12.681215
std,3.355431,2.637342,673.269446,135.049680,937.114274,5907.398399,15.459633,7.963839
min,44.302000,26.424000,0.000000,54.000000,0.065920,1.121601,0.000000,0.000000
25%,48.431250,32.979250,861.750000,158.000000,208.270516,1308.276409,11.560862,6.594610
50%,50.500500,36.162000,1557.000000,194.000000,526.568982,3479.343521,20.417158,11.539568
75%,53.191250,36.862000,1905.250000,270.000000,1114.473212,7701.447471,31.393457,17.428877
max,61.792000,39.293000,3583.000000,1142.000000,25656.226180,44108.087060,100.000000,45.000000


Analysis of numeric columns:
- `LONG` and `LAT`:
    - min and max values close to mean values
    - These two columns refer to the landslide point's position on a map, so outliers are not a concern.
- `Elevation`:
    - max value is 2.5 times larger than the mean, and also 2 times larger than the upper quartile.
    - Iran contains mountain ranges, such as the Zagros mountain range in northern Iran. With this in mind, the elevation maximum is not worrisome.
- `AAP(mm)` (average annual precipitation):
    - max value (1142) is much larger than mean (241).
    - min value (54) small, but considering Iran has an arid and semi-arid climate (meaning little rainfall), this is not strange.
    - We should look into the outliers greater than the upper quartile (270), because, as mentioned above, Iran's climate is arid.
- `RiverDIST(m)`:
    - Once again, the max value (25656) is much larger than the upper quartile (1114)
    - I would suggest that we leave these values as-is.
- `FaultDIST(m)`:
    - Both the min (1) and max (44108) are outliers
- `Slop(Percent)`:
    - The min value (0) and max value (100) coincides with the description of this column.
    - This column's values are in range of the constraints of percentage values.
- `Slop(Degrees)`:
    - Consider the description of this column: "The slope at the landslide point (Values range from 0 to 90)"
    - This column's values are in the range of the description.
    

Summary:
- Columns we should check:
    - `Elevation`
    - `AAP(mm)`
    - `RiverDIST(m)`
    - `FaultDIST(m)`
- We will only keep the data in the boundaries of the lower to upper quartile for model training. But before removing these data points, we first apply exploratory data analysis.

### Vectorising categorical text data

Some of the text columns are categorical in nature. I am vectorising these columns with the hope that we are able to calculate their correlation to numerical columns.
I will use the label encoder from SKLearn for this process.

Columns to vectorise:
- `SUB_Basin`
- `Landuse_Category`
- `GEO_UNIT`
- `Climate_Type`

In [46]:
landslide_vectorised = landslide.copy()
label_encoder_text = LabelEncoder()
landslide_vectorised['SUB_Basin_encoded'] = label_encoder_text.fit_transform(landslide_vectorised['SUB_Basin'])
landslide_vectorised['Landuse_Category_encoded'] = label_encoder_text.fit_transform(landslide_vectorised['Landuse_Category'])
landslide_vectorised['GEO_UNIT_encoded'] = label_encoder_text.fit_transform(landslide_vectorised['GEO_UNIT'])
landslide_vectorised['Climate_Type_encoded'] = label_encoder_text.fit_transform(landslide_vectorised['Climate_Type'])

landslide_vectorised.head(1)

,LONG,LAT,SUB_Basin,Elevation,AAP(mm),RiverDIST(m),FaultDIST(m),Landuse_Type,Slop(Percent),Slop(Degrees),GEO_UNIT,DES_GEOUNI,Climate_Type,DES_ClimateType,Landuse_Category,SUB_Basin_encoded,Landuse_Category_encoded,GEO_UNIT_encoded,Climate_Type_encoded
0,52.326,27.763,Mehran,617.0,137,1448.705292,40639.5789,poorrange,42.240669,22.899523,EOas-ja,"Undivided Asmari and Jahrum Formation , regard...",A-M-VW,"Warm and humid, with a humid period longer tha...",variousrange,31,9,24,3


In [47]:
# Save cleaned dataset as new .csv file
landslide.to_csv('landslide_cleaned.csv', index= False)

---
<a id="five"></a>
## **Exploratory Data Analysis (EDA)**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Explore and visualize the data to uncover patterns, trends, and relationships.
* **Details:** Use statistics and visualizations to explore the data. This may include histograms, box plots, scatter plots, and correlation matrices. Discuss any significant findings.
---


---
<a id="six"></a>
## **Modeling**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Develop and train predictive or statistical models.
* **Details:** Describe the choice of models, feature selection and engineering processes, and show how the models are trained. Include code for setting up the models and explanations of the model parameters.
---


In [ ]:
#Please use code cells to code in and do not forget to comment your code.

---
<a id="seven"></a>
## **Evaluation and Validation**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Evaluate and validate the effectiveness and accuracy of the models.
* **Details:** Present metrics used to evaluate the models, such as accuracy, precision, recall, F1-score, etc. Discuss validation techniques employed, such as cross-validation or train/test split.
---

In [ ]:
#Please use code cells to code in and do not forget to comment your code.

---
<a id="eight"></a>
## **Final Model**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Present the final model and its performance.
* **Details:** Highlight the best-performing model and discuss its configuration, performance, and why it was chosen over others.
---


In [ ]:
#Please use code cells to code in and do not forget to comment your code.

---
<a id="nine"></a>
## **Conclusion and Future Work**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Summarize the findings and discuss future directions.
* **Details:** Conclude with a summary of the results, insights gained, limitations of the study, and suggestions for future projects or improvements in methodology or data collection.
---


In [ ]:
#Please use code cells to code in and do not forget to comment your code.

---
<a id="ten"></a>
## **References**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Provide citations and sources of external content.
* **Details:** List all the references and sources consulted during the project, including data sources, research papers, and documentation for tools and libraries used.
---

Licensing information: https://creativecommons.org/licenses/by-sa/4.0/

Dataset: https://www.kaggle.com/datasets/mohammadrahdanmofrad/landslide-risk-assessment-factors

Map of Iran: https://www.cia.gov/the-world-factbook/countries/iran/map/

Map of Iran, topographic: https://mapsontheweb.zoom-maps.com/post/646909053774299136/the-topography-of-iran-by-mappdoutofficial

## Additional Sections to Consider

* ### Appendix:
For any additional code, detailed tables, or extended data visualizations that are supplementary to the main content.

* ### Contributors:
If this is a group project, list the contributors and their roles or contributions to the project.
